# Treinando um modelo Word2Vec
Neste notebook treinaremos um modelo Word2Vec a partir dos títulos de notícias. Usaremos a técnica continous bag of words e skipgram e compararemos seus resultados.
## Importando os dados 

In [1]:
import pandas as pd

In [2]:
dados_treino = pd.read_csv('treino.csv')

In [3]:
dados_teste = pd.read_csv('teste.csv')

In [4]:
dados_treino.sample()

,title,text,date,category,subcategory,link
68619,Veja o novo trailer da 3ª temporada de 'Orange...,A Netflix divulgou nesta segunda (11) um novo ...,2015-11-05,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2015/05...


# Tokenizando os dados com a biblioteca Spacy

In [6]:
import spacy

In [7]:
nlp = spacy.load("pt_core_news_sm")

In [8]:
import pt_core_news_sm

In [9]:
nlp = pt_core_news_sm.load()

In [10]:
texto = 'Rio de Janeiro é uma cidade maravilhosa'
doc = nlp(texto)

In [11]:
type(doc)

spacy.tokens.doc.Doc

In [12]:
doc.ents

(Rio de Janeiro,)

In [13]:
doc[1], doc[1].is_stop

(de, True)

In [14]:
textos_para_tratamento = dados_treino.title.str.lower()

In [15]:
def trata_textos(doc):
  tokens_validos = []
  for t in doc:
    e_valido = not t.is_stop and t.is_alpha
    if e_valido:
      tokens_validos.append(t.text)
  if len(tokens_validos) > 2:
     return ' '.join(tokens_validos)

In [16]:
trata_textos(doc)

'Rio Janeiro cidade maravilhosa'

In [17]:
type(nlp)

spacy.lang.pt.Portuguese

In [18]:
from time import time

t0 = time()
textos_tratados = [trata_textos(doc) for doc in nlp.pipe(textos_para_tratamento,
                                                        batch_size=1000,
                                                        n_process=-1)]
tf = time() - t0

In [19]:
print(tf/60)

5.017284580071768


In [20]:
textos_tratados

['polêmica marine le pen abomina negacionistas holocausto',
 'macron le pen turno frança revés siglas tradicionais',
 'apesar larga vitória legislativas macron terá desafios frente',
 'governo antecipa balanço alckmin anuncia queda homicídios sp',
 'queda maio atividade econômica sobe junho bc',
 'barcelona vence virada atlético madri bate bayern munique',
 'spartacus oferece duplo retrato batalhas perdidas',
 'sobe mortos atentado terrorista nice frança',
 'premiada sundance crystal moselle retrata sexismo mundo skate',
 'metroviários ferroviários ameaçam parar terça paulo',
 'anos angeli tirinhas diárias ilustrada',
 'mortes acidentes trânsito caem cidade sp',
 'daniel craig será stormtrooper star wars ator',
 'justiça sp libera construção moradias manancial billings',
 'prefiro urnas julguem lula achem batom cueca',
 'olivier anquier transformar cobertura centro restaurante',
 'mvp nba temporadas steve nash anuncia aposentadoria',
 'frança nega vinculo terrorista ataque soldados mes

In [21]:
titulos_tratados = pd.DataFrame({'titulo': textos_tratados})
titulos_tratados.head()

,titulo
0,polêmica marine le pen abomina negacionistas h...
1,macron le pen turno frança revés siglas tradic...
2,apesar larga vitória legislativas macron terá ...
3,governo antecipa balanço alckmin anuncia queda...
4,queda maio atividade econômica sobe junho bc


## O modelo - treinando com continuous bag of words

In [22]:
from gensim.models import Word2Vec

w2v_modelo = Word2Vec(sg=0,
                      window=2,
                      vector_size=300,
                      min_count=5,
                      alpha=0.3,
                      min_alpha=0.007)

In [23]:
print(len(titulos_tratados))

titulos_tratados = titulos_tratados.dropna().drop_duplicates()

print(len(titulos_tratados))

90000
84466


In [24]:
lista_lista_tokens = [titulo.split(' ') for titulo in titulos_tratados.titulo]

In [25]:
lista_lista_tokens

[['polêmica', 'marine', 'le', 'pen', 'abomina', 'negacionistas', 'holocausto'],
 ['macron', 'le', 'pen', 'turno', 'frança', 'revés', 'siglas', 'tradicionais'],
 ['apesar',
  'larga',
  'vitória',
  'legislativas',
  'macron',
  'terá',
  'desafios',
  'frente'],
 ['governo',
  'antecipa',
  'balanço',
  'alckmin',
  'anuncia',
  'queda',
  'homicídios',
  'sp'],
 ['queda', 'maio', 'atividade', 'econômica', 'sobe', 'junho', 'bc'],
 ['barcelona',
  'vence',
  'virada',
  'atlético',
  'madri',
  'bate',
  'bayern',
  'munique'],
 ['spartacus', 'oferece', 'duplo', 'retrato', 'batalhas', 'perdidas'],
 ['sobe', 'mortos', 'atentado', 'terrorista', 'nice', 'frança'],
 ['premiada',
  'sundance',
  'crystal',
  'moselle',
  'retrata',
  'sexismo',
  'mundo',
  'skate'],
 ['metroviários', 'ferroviários', 'ameaçam', 'parar', 'terça', 'paulo'],
 ['anos', 'angeli', 'tirinhas', 'diárias', 'ilustrada'],
 ['mortes', 'acidentes', 'trânsito', 'caem', 'cidade', 'sp'],
 ['daniel', 'craig', 'será', 'stormt

In [42]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

w2v_modelo = Word2Vec(sg=0,
                      window=2,
                      vector_size=300,
                      min_count=5)

w2v_modelo.build_vocab(lista_lista_tokens, progress_per=5000)

In [28]:
(dir(w2v_modelo))

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adapt_by_suffix',
 '_check_corpus_sanity',
 '_check_training_sanity',
 '_clear_post_train',
 '_do_train_epoch',
 '_do_train_job',
 '_get_next_alpha',
 '_get_thread_working_mem',
 '_job_producer',
 '_load_specials',
 '_log_epoch_end',
 '_log_epoch_progress',
 '_log_progress',
 '_log_train_end',
 '_raw_word_count',
 '_save_specials',
 '_scan_vocab',
 '_smart_save',
 '_train_epoch',
 '_train_epoch_corpusfile',
 '_worker_loop',
 '_worker_loop_corpusfile',
 'add_lifecycle_event',
 'add_null_word',
 'alpha',
 'batch_words',
 'build_vocab',
 'build_vocab_from_freq',
 'cbow_mean',
 'comment',
 'compute_loss',
 'corpus_count',
 

In [29]:
w2v_modelo.corpus_count

84466

In [30]:
from gensim.models.callbacks import CallbackAny2Vec
from gensim.models import Word2Vec

# iniciando a chamada callback
class callback(CallbackAny2Vec):
  def __init__(self):
    self.epoch = 0

  def on_epoch_end(self, model):
    loss = model.get_latest_training_loss()
    if self.epoch == 0:
        print('LossA após a época {}: {:.2f} {}'.format(self.epoch, loss, type(loss)))
    else:
        print('LossB após a época {}: {:.2f}'.format(self.epoch, loss- self.loss_previous_step))
    self.epoch += 1
    self.loss_previous_step = loss


In [31]:
w2v_modelo.train(lista_lista_tokens,
                 total_examples=w2v_modelo.corpus_count,
                 epochs=30,
                 compute_loss=True,
                 callbacks=[callback()])

LossA após a época 0: 653562.31 <class 'float'>
LossB após a época 1: 509959.69
LossB após a época 2: 476875.38
LossB após a época 3: 460719.12
LossB após a época 4: 302610.50
LossB após a época 5: 357280.25
LossB após a época 6: 337213.75
LossB após a época 7: 267636.00
LossB após a época 8: 285503.00
LossB após a época 9: 304440.75
LossB após a época 10: 257729.75
LossB após a época 11: 221903.50
LossB após a época 12: 201073.50
LossB após a época 13: 216805.50
LossB após a época 14: 188399.00
LossB após a época 15: 193521.50
LossB após a época 16: 186719.00
LossB após a época 17: 170927.00
LossB após a época 18: 177592.50
LossB após a época 19: 162559.50
LossB após a época 20: 178074.00
LossB após a época 21: 156179.50
LossB após a época 22: 161325.00
LossB após a época 23: 158803.00
LossB após a época 24: 164351.00
LossB após a época 25: 171017.50
LossB após a época 26: 160571.50
LossB após a época 27: 133584.00
LossB após a época 28: 133090.00
LossB após a época 29: 156402.00


(14584408, 16207260)

In [32]:
w2v_modelo.wv.most_similar('google')

[('apple', 0.6951244473457336),
 ('facebook', 0.6545721292495728),
 ('amazon', 0.6024664640426636),
 ('airbnb', 0.6004083156585693),
 ('yahoo', 0.5908020734786987),
 ('reputação', 0.5870165824890137),
 ('volkswagen', 0.5567417740821838),
 ('sony', 0.5470452904701233),
 ('software', 0.5434287190437317),
 ('alibaba', 0.543320894241333)]

In [33]:
w2v_modelo.wv.most_similar('barcelona')

[('barça', 0.6775088906288147),
 ('psg', 0.6685528755187988),
 ('bayern', 0.6668044328689575),
 ('chelsea', 0.6660465598106384),
 ('juventus', 0.6536033153533936),
 ('madrid', 0.6513886451721191),
 ('betis', 0.6391227841377258),
 ('united', 0.63421630859375),
 ('munique', 0.6180722117424011),
 ('suárez', 0.6115201115608215)]

In [34]:
w2v_modelo.wv.most_similar('messi')

[('suárez', 0.697401225566864),
 ('cavani', 0.6293045878410339),
 ('tevez', 0.620309054851532),
 ('ronaldo', 0.6143797636032104),
 ('barcelona', 0.6022230982780457),
 ('neymar', 0.6020382642745972),
 ('benzema', 0.6000772714614868),
 ('cristiano', 0.5985957384109497),
 ('chuteiras', 0.5890712738037109),
 ('enrique', 0.5869767665863037)]

In [35]:
w2v_modelo.wv.most_similar('gm')

[('embraer', 0.7577744126319885),
 ('chrysler', 0.7242559194564819),
 ('braskem', 0.7234333753585815),
 ('volks', 0.7215813398361206),
 ('csn', 0.7061935067176819),
 ('distribuidora', 0.686822772026062),
 ('honda', 0.6855748891830444),
 ('metalúrgicos', 0.6759077906608582),
 ('volkswagen', 0.6713950037956238),
 ('usiminas', 0.65245121717453)]

## Treinando com skipgram

In [36]:
# treinamento do modelo skip-gram
w2v_modelo_sg = Word2Vec(sg=1,
                        window=5,
                        vector_size=300,
                        min_count=5)

w2v_modelo_sg.build_vocab(lista_lista_tokens,
                          progress_per=5000)

w2v_modelo_sg.train(lista_lista_tokens,
                    total_examples=w2v_modelo_sg
                                    .corpus_count,
                    epochs=30,
                    compute_loss=True,
                    callbacks=[callback()])

LossA após a época 0: 1954055.88 <class 'float'>
LossB após a época 1: 1495135.12
LossB após a época 2: 1265587.00
LossB após a época 3: 1164675.50
LossB após a época 4: 1072420.00
LossB após a época 5: 1092326.00
LossB após a época 6: 922433.50
LossB após a época 7: 853716.00
LossB após a época 8: 833344.00
LossB após a época 9: 816498.00
LossB após a época 10: 841820.00
LossB após a época 11: 821273.00
LossB após a época 12: 760773.00
LossB após a época 13: 744952.00
LossB após a época 14: 770697.00
LossB após a época 15: 756466.00
LossB após a época 16: 680590.00
LossB após a época 17: 554668.00
LossB após a época 18: 510242.00
LossB após a época 19: 494052.00
LossB após a época 20: 481766.00
LossB após a época 21: 469882.00
LossB após a época 22: 461278.00
LossB após a época 23: 447450.00
LossB após a época 24: 464508.00
LossB após a época 25: 427132.00
LossB após a época 26: 401646.00
LossB após a época 27: 415650.00
LossB após a época 28: 410770.00
LossB após a época 29: 428100.0

(14584163, 16207260)

In [37]:
w2v_modelo_sg.wv.most_similar('google')

[('reguladores', 0.47857239842414856),
 ('android', 0.4753905236721039),
 ('waze', 0.464324951171875),
 ('apple', 0.4573425352573395),
 ('buffett', 0.45178550481796265),
 ('yahoo', 0.4369227886199951),
 ('toshiba', 0.43672919273376465),
 ('walmart', 0.4361095428466797),
 ('app', 0.43604645133018494),
 ('chips', 0.42875564098358154)]

In [38]:
w2v_modelo.wv.most_similar('google')

[('apple', 0.6951244473457336),
 ('facebook', 0.6545721292495728),
 ('amazon', 0.6024664640426636),
 ('airbnb', 0.6004083156585693),
 ('yahoo', 0.5908020734786987),
 ('reputação', 0.5870165824890137),
 ('volkswagen', 0.5567417740821838),
 ('sony', 0.5470452904701233),
 ('software', 0.5434287190437317),
 ('alibaba', 0.543320894241333)]

In [39]:
w2v_modelo_sg.wv.most_similar('gm')

[('metalúrgicos', 0.6404317021369934),
 ('motors', 0.6276149153709412),
 ('montadora', 0.5920149087905884),
 ('honda', 0.5850511193275452),
 ('audi', 0.5833036303520203),
 ('compartilhamento', 0.5688266158103943),
 ('chrysler', 0.5676087141036987),
 ('bmw', 0.5646654963493347),
 ('airbags', 0.5589927434921265),
 ('toyota', 0.5583388209342957)]

In [40]:
w2v_modelo.wv.most_similar('gm')

[('embraer', 0.7577744126319885),
 ('chrysler', 0.7242559194564819),
 ('braskem', 0.7234333753585815),
 ('volks', 0.7215813398361206),
 ('csn', 0.7061935067176819),
 ('distribuidora', 0.686822772026062),
 ('honda', 0.6855748891830444),
 ('metalúrgicos', 0.6759077906608582),
 ('volkswagen', 0.6713950037956238),
 ('usiminas', 0.65245121717453)]

w2v_modelo.wv.save_word2vec_format('/content/drive/MyDrive/Word2Vec interpretação da linguagem humana com Word embedding/modelo_cbow.txt', binary=False)

w2v_modelo_sg.wv.save_word2vec_format('/content/drive/MyDrive/Word2Vec interpretação da linguagem humana com Word embedding/modelo_skipgram.txt', binary=False)